# Data pre-processing to make it ready for the model to learn



#### Have you identified other variables transformations that you need to do based on the distributions you have analyzed so far?

In [ ]:
#Feature engineering is the process of creating features (also called "attributes") 
#that don't already exist in the dataset. This means that if your dataset already contains enough 
#"useful" features, you don't necessarily need to engineer additional features.

Answer here

In [10]:
import math
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn import linear_model
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter(action='ignore', category=Warning)
np.seterr(divide='ignore', invalid='ignore')

# Mount local files
#from google.colab import drive
#drive.mount('/gdrive')

# Hide deprecated warnings
import warnings
warnings.filterwarnings('ignore')

In [11]:
#  Load the clean dataset form previous notebook

In [12]:
# Load data from the folder where I uploaded the csv file data with grouped Features
data = pd.read_csv('/Users/AirMorena/Desktop/final_proj/csv/google_play_store/data_engineer1.csv')

# define functions

In [13]:
def one_hot_encode_categorical_variables(series, name):
      return pd.get_dummies(series, prefix=name)

In [14]:
# Check the dimensions
data.shape

(9351, 27)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9351 entries, 0 to 9350
Data columns (total 27 columns):
App                         9351 non-null object
Category                    9351 non-null object
Rating                      9351 non-null float64
Reviews                     9351 non-null int64
Size                        9351 non-null float64
Installs                    9351 non-null int64
Type                        9351 non-null object
Price                       9351 non-null float64
Content Rating              9351 non-null object
Genres                      9351 non-null object
Last Updated                9351 non-null object
Current Ver                 9351 non-null object
Android Ver                 9351 non-null float64
day                         9351 non-null int64
month                       9351 non-null int64
year                        9351 non-null int64
month_year                  9351 non-null object
Current Ver_clean           9351 non-null float64
App_dup    

In [16]:
# if there are missing values: is there at least one missing  value? True or false
#data.isnull().sum()

## Dummify categorical variables

* We want our categorical variables to be part of our logistic regression model. We need to transform into boolean features?

In [17]:
data.columns.tolist()

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver',
 'day',
 'month',
 'year',
 'month_year',
 'Current Ver_clean',
 'App_dup',
 'Successful_App',
 'Installs_group',
 'reviews_levels',
 'Size_group',
 'Current Ver_clean_levels',
 'Android Ver_levels',
 'st_Rating',
 'st_Price']

In [9]:
#data.head()

In [18]:
# drop columns we will not use for the model: (the continous , since we create categorical from them)
data1=data.drop(['App','Size','Rating','Installs', 'st_Rating','Reviews','st_Price','Android Ver','Price','Current Ver','Current Ver_clean','Rating',
                 'Price','Last Updated','Genres','day','month_year','st_Price','App_dup','Installs_group'], axis=1)

In [19]:
data1.columns.tolist()

['Category',
 'Type',
 'Content Rating',
 'month',
 'year',
 'Successful_App',
 'reviews_levels',
 'Size_group',
 'Current Ver_clean_levels',
 'Android Ver_levels']

In [20]:
data1.shape
# 9351 entries and 10 features

(9351, 10)

In [21]:
data1.head()

,Category,Type,Content Rating,month,year,Successful_App,reviews_levels,Size_group,Current Ver_clean_levels,Android Ver_levels
0,ART_AND_DESIGN,Free,Everyone,1,2018,0,low,small,low,high
1,ART_AND_DESIGN,Free,Everyone,1,2018,0,low,small,medium,high
2,ART_AND_DESIGN,Free,Everyone,8,2018,1,high,small,low,high
3,ART_AND_DESIGN,Free,Teen,6,2018,1,high,big,high,high
4,ART_AND_DESIGN,Free,Everyone,6,2018,0,low,small,low,high


In [128]:
# feature engineering

In [22]:
# Transform categorical variables into boolean new columns with pandas get_dummies function
#total columns minus target variable 'Successful_App', and minus 'App' minus installs groups and rating, and price
total_columns={
'Category',
 'Type',
 'Content Rating',
 'month',
 'year',
 'reviews_levels',
 'Size_group',
 'Current Ver_clean_levels',
 'Android Ver_levels'
}

new_boolean_columns = []
for feature in total_columns:
    new_boolean_columns.append(one_hot_encode_categorical_variables(data1[feature], feature))

In [24]:
#new_boolean_columns

In [26]:
#new_boolean_columns

In [27]:
#pd.concat(new_boolean_columns, axis=1) 

In [28]:
df_new_boolean_columns = pd.concat(new_boolean_columns, axis=1)  
#Drop old categorical features
data2 = data1.drop(total_columns, axis=1)

#Append dummy categorical features (now boolean)
final_feature_vector = pd.concat([data2['Successful_App'], 
                                  df_new_boolean_columns], axis=1)

In [29]:
final_feature_vector.head() 

,Successful_App,Type_Free,Type_Paid,Size_group_big,Size_group_small,month_1,month_2,month_3,month_4,month_5,...,Category_WEATHER,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [30]:
final_feature_vector.columns

Index(['Successful_App', 'Type_Free', 'Type_Paid', 'Size_group_big',
       'Size_group_small', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12', 'Current Ver_clean_levels_high',
       'Current Ver_clean_levels_low', 'Current Ver_clean_levels_medium',
       'reviews_levels_high', 'reviews_levels_low', 'reviews_levels_medium',
       'Android Ver_levels_high', 'Android Ver_levels_low',
       'Content Rating_Adults only 18+', 'Content Rating_Everyone',
       'Content Rating_Everyone 10+', 'Content Rating_Mature 17+',
       'Content Rating_Teen', 'Content Rating_Unrated',
       'Category_ART_AND_DESIGN', 'Category_AUTO_AND_VEHICLES',
       'Category_BEAUTY', 'Category_BOOKS_AND_REFERENCE', 'Category_BUSINESS',
       'Category_COMICS', 'Category_COMMUNICATION', 'Category_DATING',
       'Category_EDUCATION', 'Category_ENTERTAINMENT', 'Category_EVENTS',
       'Category_FAMILY', 'Categ

### Get the final shape of your feature vector dataframe

In [31]:
final_feature_vector.shape

(9351, 73)

## Store the data into a new csv file

In [32]:
final_feature_vector.to_csv("/Users/AirMorena/Desktop/final_proj/csv/google_play_store/170619final_feature_vector.csv", index=False, encoding='utf-8')

In [33]:
final_feature_vector.head()

,Successful_App,Type_Free,Type_Paid,Size_group_big,Size_group_small,month_1,month_2,month_3,month_4,month_5,...,Category_WEATHER,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
